In [113]:
import pandas as pd
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON
import json
import math

In [111]:
def getList(dict):
    list = []
    for key in dict.keys():
        list.append(key)

    return list


k = 4
user = 29
data = pd.read_csv("data.csv")
daycontext = pd.read_csv("context_day.csv")
placecontext = pd.read_csv("context_place.csv")
movieNames = pd.read_csv('movie_names.csv')
weekend = [' Sat', ' Sun']
place = [' h']

#подготовка словаря для получения коэф. схожести
forSimData = data[data['Unnamed: 0'] != 'User 30'].T

del data['Unnamed: 0']
chosenUser = data.iloc[user]
simDict = dict()
for i in forSimData:
    simDict[i] = 0

#подсчет коэф. сходства
for i in simDict.keys():
    sumuv = 0
    sumu2 = 0
    sumv2 = 0
    movie = 0
    for item in data.iloc[i]:
        if(chosenUser[movie] != -1 and item != -1):
            sumuv += chosenUser[movie]*item
            sumu2 += chosenUser[movie]**2
            sumv2 += item**2
            simDict[i] = sumuv / (math.sqrt(sumu2) * math.sqrt(sumv2))
        movie+=1


#результат 1 - похожие пользователи
result1 = dict(sorted(simDict.items(), key = lambda kv:kv[1], reverse = True) [:k])
print('Похожие пользователи:')
print(result1)


toMeanCounter = 0
cnt = 0
meanCurrentUser = 0
meanDict = dict()
#подсчет средней оценки текущего пользователя
for i in chosenUser:
    if i>=0:
        toMeanCounter+=i
        cnt+=1

meanCurrentUser = toMeanCounter/cnt
#подсчет средних оценок похожих пользователей
for i in result1:
    toMeanCounter = 0
    cnt = 0
    for j in data.iloc[i]:
        if j >= 0:
            toMeanCounter += j
            cnt += 1
    meanDict[i] = toMeanCounter/cnt


shouldRate = list()
#фильмы, которые пользователь не оценил
counter = 0
for i in chosenUser:
    if i == -1:
        shouldRate.append(counter)
    counter += 1

result1Task = dict()
for film in shouldRate:
    sumup = 0
    sumdown = 0
    for i in meanDict:
        if(data.iloc[i][film] != -1):
            sumup += simDict[i]*(data.iloc[i][film]-meanDict[i])
            sumdown += simDict[i]

    result1Task[film+1] = round(meanCurrentUser + sumup/sumdown, 3)
print('Фильмы, которые пользователь не оценил, с предполагаемой оценкой:')
#не использую контекстную оценку, т.к. для моего пользователя нет фильмов, которые можно посмотреть в сб-вс
print(result1Task)
print('Названия фильмов:')

movies = result1Task.keys()
names = []
for m in movies:
    names.append(movieNames.iloc[m-1][0])


#удаляю мусор из строк
#вынужден обрубать название фильма, т.к. не находит полностью
updatednames = list()
for name in names:
    name = name[:name.find("-")]
    name = name.rstrip(';')
    name = name.lstrip(' ')
    updatednames.append(name)
print(updatednames)

Похожие пользователи:
{6: 0.8981521691677541, 28: 0.8958770163450452, 34: 0.8834766400145097, 4: 0.8777740396442079}
Фильмы, которые пользователь не оценил, с предполагаемой оценкой:
{13: 3.54, 14: 2.811}
Названия фильмов:
['Star Wars: Episode V ', 'Inception']


In [114]:
#поиск id для запроса
#использую запрос для поиска именно фильмов (не находит inception просто так)
idfromwiki = dict()

API = "https://www.wikidata.org/w/api.php"
for name in updatednames:  
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql_query = """ SELECT ?movie
    WHERE {
    ?movie wdt:P31 wd:Q11424;
    wdt:P1476 ?title filter regex(?title,\"""" + name + """\",'i').
    }
    """
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    idfromwiki[name] = str(results['results']['bindings'][0]['movie']['value']).replace('http://www.wikidata.org/entity/', '')
   
print(idfromwiki)

{'Star Wars: Episode V ': 'Q181795', 'Inception': 'Q25188'}


In [115]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
endRes = dict()
for [name, wikiId] in list(idfromwiki.items()):
    
    sparql_query = """
    SELECT ?actor ?actorLabel
        WHERE {
        ?film wdt:P161 ?actor.
        ?actor wdt:P21 ?gender.
        FILTER(?film = wd:""" + wikiId + """&& ?gender = wd:Q6581072)
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".}

    MINUS{

        SELECT ?actor  ?actorLabel 
        WHERE{
          ?film wdt:P161 ?actor.
          ?film wdt:P577 ?date.
          FILTER(?date < ?dateOfChosen)
          {
          SELECT ?actor ?actorLabel ?dateOfChosen
              WHERE {
                ?film wdt:P161 ?actor.
                ?actor wdt:P21 ?gender.
                ?film wdt:P577 ?dateOfChosen.
                FILTER(?film = wd:""" + wikiId + """ && ?gender = wd:Q6581072)
                SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".}
                }
              }
            }
        }
    }
    """

    sparql.setQuery(sparql_query)

    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = dict()
    results_df = results['results']['bindings']
    if len(results_df) > 0:
        endRes[name] = results_df[0]['actorLabel']['value']
    else:
        endRes[name] = []
print(endRes)

{'Star Wars: Episode V ': [], 'Inception': []}
